In [1]:
import numpy as np
import pandas

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import csv

In [2]:
from gpmaniflow.surfaces import BernsteinPolynomial
from gpmaniflow.utils import GetAllListPairs
import gpflow
import tensorflow as tf

2022-02-15 22:52:41.780406: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-15 22:52:41.780430: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
order = 10
X = np.linspace(0.0, 1., order + 1).reshape(order + 1, 1)
print(X)
B = BernsteinPolynomial(orders = order)
BX = B(X) 
MB = BX ** 2
M = tf.squeeze(MB)
print(M)

[[0. ]
 [0.1]
 [0.2]
 [0.3]
 [0.4]
 [0.5]
 [0.6]
 [0.7]
 [0.8]
 [0.9]
 [1. ]]
tf.Tensor(
[[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.21576655e-01 1.50094635e-01 3.75236588e-02 3.29425811e-03
  1.24551426e-04 2.21424757e-06 1.89836040e-08 7.65275040e-11
  1.32860250e-13 8.10000000e-17 1.00000000e-20]
 [1.15292150e-02 7.20575940e-02 9.11978925e-02 4.05323966e-02
  7.75815405e-03 6.98233864e-04 3.03052892e-05 6.18475291e-07
  5.43581798e-09 1.67772160e-11 1.04857600e-14]
 [7.97922663e-04 1.46557224e-02 5.45103144e-02 7.11971453e-02
  4.00483942e-02 1.05923916e-02 1.35107036e-03 8.10304589e-05
  2.09294234e-06 1.89836040e-08 3.48678440e-11]
 [3.65615844e-05 1.62495931e-03 1.46246338e-02 4.62210647e-02
  6.29120048e-02 4.02636830e-02 1.24270627e-02 1.80347395e-03
  1.12717122e-04 2.47390116e-06 1.09951163e-08]
 [9.53674316e-07 9.53674316e-05 1.93119049e-03 1.37

2022-02-15 22:52:43.487964: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-15 22:52:43.488013: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-15 22:52:43.488053: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (martinj): /proc/driver/nvidia/version does not exist
2022-02-15 22:52:43.488681: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
Minv = np.linalg.inv(M.numpy())
P = Minv @ np.array([[1] * (order + 1)]).T
print(np.sqrt(P))

[[1.        ]
 [2.18291113]
 [1.94179959]
 [2.53205321]
 [2.16801851]
 [2.61163579]
 [2.16801851]
 [2.53205321]
 [1.94179959]
 [2.18291113]
 [1.        ]]


In [5]:
I = GetAllListPairs(order + 1, 1)

outB = tf.gather(BX, I, axis = 2)
outB = tf.linalg.diag_part(tf.transpose(outB, perm = (0,2,1,3)))
outB = tf.reduce_prod(outB, axis = 2)

f_var = tf.matmul(outB ** 2, P)
print(f_var)

tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(11, 1), dtype=float64)
